In [ ]:
import sys
import os
import numpy as np
import pandas as pd

import matplotlib.cm as cm
import matplotlib.pyplot as plt
from colorcet import palette

from datetime import datetime

In [ ]:
from tqdm import tqdm

In [ ]:
filename = 'primes_to_{0}.dat'.format(points)
figsave_format = 'png'

In [ ]:
points = 2**26
print('According to prime theorem, we should find approximately {0} prime numbers.'.format(points // np.log(points)))

In [ ]:
start = datetime.now()
# Producing prime numbers
if filename not in os.listdir('./data'):
    lower_lim = 2
    upper_lim = points

    prime_nums = []
    for n in tqdm(range(lower_lim, upper_lim+1),):
        for i in range(2, int(np.ceil(np.sqrt(n)))):
            if (n % i) == 0:
                break
        else:
            prime_nums.append(n)

    np.savetxt(fname='./data/' + filename, X=prime_nums)

else:
    prime_nums = np.genfromtxt('./data/' + filename)

# Producing natural numbers
    natural_nums = [i for i in range(points)]

prime_nums = np.array(prime_nums)
print('Finished searching prime numbers up to {0} in {1}'.format(points, str(datetime.now() - start)))
print('Found prime numbers : {0}'.format(len(prime_nums)))

In [ ]:
def polar_primes(lim=None, scattersize=10, save=False):

    if lim is not None:
        lim = int(lim)
    
    nrows = 1
    ncols = 1
    figscale = 150
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*figscale,nrows*figscale),
                             subplot_kw={'projection': 'polar'}, facecolor='black')

    axes.axis('off')
    axes.set_facecolor('black')
    
    x_coords = prime_nums[0:lim] * np.cos(prime_nums[0:lim])
    y_coords = prime_nums[0:lim] * np.sin(prime_nums[0:lim])
    plot_range = np.max([x_coords, y_coords]) * 2/3
    axes.set_ylim(0, plot_range)
    #axes.scatter(natural_nums, natural_nums, s=scattersize, color='tab:blue')
    
    colors = []
    # cm.autumn_r
    cmap = cm.hot_r
    NUM_COLORS = len(prime_nums[0:lim])
    for i in range(NUM_COLORS):
        colors.append(cmap(1.*i/NUM_COLORS))  # color will now be an RGBA tuple
    axes.scatter(prime_nums[0:lim], prime_nums[0:lim],
                 s=scattersize, color=colors)

    if save:
        plt.savefig('out/primes_{0}.'.format(len(prime_nums[0:lim])) + figsave_format,
                    format=figsave_format,
                    dpi=None,
                    facecolor='black',
                    bbox_inches='tight')
    #plt.show()
    plt.ioff()
    plt.close()

In [ ]:
polar_primes(lim=1e03, scattersize=400, save=True)

In [ ]:
polar_primes(lim=1e04, scattersize=200, save=True)

In [ ]:
polar_primes(lim=None, scattersize=1, save=True)

## Datashader

In [ ]:
import datashader as ds, pandas as pd
import datashader.transfer_functions as tf
import datashader.utils as utils

In [ ]:
def ds_polar_primes(width, lim=None):

    width = int(width)
    if lim is not None:
        lim = int(lim)

    # Convert to polar coordinates
    prime_polar = prime_nums[0:lim] * np.array([np.cos(prime_nums[0:lim]), np.sin(prime_nums[0:lim])])
    #plot_range = np.max([prime_polar[0], prime_polar[1]])
    # Create dataframe for Datashader points plot
    df = pd.DataFrame({'x':pd.Series(prime_polar[0]),'y':pd.Series(prime_polar[1])})
    # Selected cmap for plot
    cmap = palette.fire

    cvs = ds.Canvas(plot_width=width, plot_height=width)
    agg = cvs.points(df,'x','y')
    img = tf.set_background(tf.shade(agg, cmap=cmap),'black')
    utils.export_image(img=img, filename=('./out/ds_primes_{0}'.format(len(prime_nums[0:lim]))))
    
    return img

In [ ]:
ds_polar_primes(width=2000, lim=2e25)

In [ ]:
ds_polar_primes(width=2**11, lim=1e04)

In [ ]:
ds_polar_primes(width=2**11, lim=None)